In [14]:
import os, time
import pandas as pd
import pymysql
import numpy as np
from tqdm.auto import tqdm

def dev_db_conn():
    """개발 DB 접속 함수"""
    conn = pymysql.connect(
        host = 'db-7ma06.pub-cdb.ntruss.com',
        user = 'redtable',
        password = 'fpemxpdlqmf5491!@#',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    return conn


def rt_db_conn():
    """빅데이터센터 DB 접속 함수"""
    conn = pymysql.connect(
        host = '220.95.212.6',
        user = 'red_dev_jiang',
        password = 'redtable211b',
        db = 'redtable_bigdata_center',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    conn.query("set character_set_connection=utf8;")
    conn.query("set character_set_server=utf8;")
    conn.query("set character_set_client=utf8;")
    conn.query("set character_set_results=utf8;")
    conn.query("set character_set_database=utf8;")
    return conn


file_path = os.path.join(os.environ["HOMEPATH"], "Downloads/")

# 상권분석정보

# RT_RESTAURANT_COMMERCIAL_AREA_INFO_202407

## https://www.bigdata-culture.kr/bigdata/user/data_market/detail.do?id=6999f161-fe7d-40be-bb8a-05fed62cc77c

In [2]:
with rt_db_conn() as conn:
    sql = """
    SELECT TABLE_NAME, COLUMN_NAME, column_comment
    FROM information_schema.`COLUMNS`
    WHERE TABLE_NAME = 'RT_RESTAURANT_COMMERCIAL_AREA_INFO_202407'
    """
    columns = pd.read_sql(sql, conn)
    
# 컬럼명 가져오기
db_cols = ','.join(columns['COLUMN_NAME'])

In [10]:
df = pd.read_excel(f"{file_path}대한민국 매출 100대 상권.xlsx", sheet_name="2021_대한민국_100대상권")
df = df[df["LDMARK_MASTER_ID"] != '-']
df = df.astype({"LDMARK_MASTER_ID":"int"})
df.reset_index(drop=True, inplace=True)

In [11]:
df

,순위,상권명,LDMARK_MASTER_ID,LDMARK_MASTER_NM,LDMARK_MASTER_TYPE,최종상권명
0,1.0,서울 암구정역,9352,압구정역,지하철역,서울 암구정역
1,2.0,서울 강남역 북부,9305,강남역,지하철역,서울 강남역
2,3.0,서울 강남역 남부,9305,강남역,지하철역,서울 강남역
3,4.0,서울 신사 ~ 논현역,9353,신사역,지하철역,서울 신사 ~ 논현역
4,4.0,서울 신사 ~ 논현역,9507,논현역,지하철역,서울 신사 ~ 논현역
...,...,...,...,...,...,...
90,96.0,서울 건대입구역,9295,건대입구역,지하철역,서울 건대입구역
91,97.0,울산 삼산동,36053,태화강역,지하철역,울산 삼산동
92,98.0,부산역 인근,9999,부산역,지하철역,부산역 인근
93,99.0,포항 상대동 상권,28579,포항문화예술회관,관광지,포항 상대동 상권


In [15]:
for i in tqdm(range(len(df))):
    with dev_db_conn() as conn:
        with conn.cursor() as curs:
            sql = """
            SELECT REPLACE(REPLACE(REPLACE(T1.AREA_NM_1, '강원도', '강원특별자치도'), '전라북도', '전북특별자치도'), '제주도', '제주특별자치도') AS 'CTY_NM',
                T3.SRC_ID AS 'RSTRNT_ID', T1.RSTR_NM AS 'RSTRNT_NM',
                T1.RSTR_RDNMADR AS 'RSTRNT_ROAD_NM_ADDR', %s AS 'RSTRNT_TRDAR_NM'
            FROM redtable01.RSTR T1
            INNER JOIN redtable01.RSTR_INDEX T2 ON T1.ID = T2.RSTR_ID
            INNER JOIN redtable01.RSTR_CONN T3 ON T2.RSTR_ID = T3.RSTR_ID
            LEFT JOIN redtable01.RSTR_LDMARK T4 ON T3.RSTR_ID = T4.RSTR_ID
            LEFT JOIN redtable01.LDMARK_MASTER T5 ON T4.LDMARK_MASTER_ID = T5.ID
            WHERE T2.IS_USE=1 AND T2.IS_OPEN=1 AND T3.SRC='live_db' AND T1.AREA_NM_1 IS NOT NULL AND T1.AREA_NM_1<>''
              AND T1.RSTR_NM IS NOT NULL AND T1.RSTR_NM<>'' AND T1.RSTR_NM NOT LIKE '%%？%%'
              AND T1.RSTR_RDNMADR IS NOT NULL AND T1.RSTR_RDNMADR<>''
              AND T1.ID IN (SELECT RSTR_ID FROM redtable01.RSTR_LDMARK WHERE LDMARK_MASTER_ID = %s)
            GROUP BY T1.ID
            """
            curs.execute(sql, (df["최종상권명"][i], df["LDMARK_MASTER_ID"][i]) )
            df_result = pd.DataFrame(curs.fetchall())

    if i == 0:
        df_final = df_result

    else:
        df_final = pd.concat([df_final, df_result])

    time.sleep(2)

  0%|          | 0/95 [00:00<?, ?it/s]

In [16]:
# 결측치 및 불필요한 데이터 정리
df_final = df_final.fillna(np.nan)
df_final.replace({np.nan:None, '\\n':None}, inplace=True)

In [17]:
df_final

,CTY_NM,RSTRNT_ID,RSTRNT_NM,RSTRNT_ROAD_NM_ADDR,RSTRNT_TRDAR_NM
0,서울특별시,103210,탠저린,서울특별시 강남구 논현로146길 9,서울 암구정역
1,서울특별시,127711,쇼토,서울특별시 강남구 압구정로18길 14-6,서울 암구정역
2,서울특별시,943071,뜸들이다,서울특별시 강남구 논현로167길 16,서울 암구정역
3,서울특별시,1180116,노커피서울,서울특별시 강남구 압구정로20길 19,서울 암구정역
4,서울특별시,1180865,베이크(BAKE)-한시적,서울특별시 강남구 압구정로 165,서울 암구정역
...,...,...,...,...,...
14,서울특별시,1186630,작수불,서울특별시 영등포구 당산로18길 9-7,서울 영등포구청역 인근
15,서울특별시,1187632,케익 앤 팜,서울특별시 영등포구 영등포로28길 5,서울 영등포구청역 인근
16,서울특별시,1187646,코젤다크하우스(kozel DARK HAUS),서울특별시 영등포구 국회대로34길 15-1,서울 영등포구청역 인근
17,서울특별시,1189019,스팀피쳐,서울특별시 영등포구 당산로 166,서울 영등포구청역 인근


In [18]:
with rt_db_conn() as conn:
    with conn.cursor() as curs:
        sql = f"""
        INSERT IGNORE INTO redtable_bigdata_center.RT_RESTAURANT_COMMERCIAL_AREA_INFO_202407({db_cols})
        VALUES (%s,%s,%s,%s,%s)
        """
        val = df_final.apply(lambda row:(row["CTY_NM"], row["RSTRNT_ID"], row["RSTRNT_NM"], row["RSTRNT_ROAD_NM_ADDR"],
                                  row["RSTRNT_TRDAR_NM"]), axis=1).tolist()
        curs.executemany(sql, val)